In [1]:
import cv2,os

data_path='Videos/facemask/face-mask-detection-keras-master/dataset/'
categories=os.listdir(data_path)
labels=[i for i in range(len(categories))]

label_dict=dict(zip(categories,labels)) #empty dictionary

print(label_dict)
print(categories)
print(labels)

{'with_mask': 0, 'without_mask': 1}
['with_mask', 'without_mask']
[0, 1]


In [2]:
img_size=100
data=[]
target=[]


for category in categories:
    folder_path=os.path.join(data_path,category)
    img_names=os.listdir(folder_path)
        
    for img_name in img_names:
        img_path=os.path.join(folder_path,img_name)
        img=cv2.imread(img_path)

        try:
            gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)           
            resized=cv2.resize(gray,(img_size,img_size))
            data.append(resized)
            target.append(label_dict[category])
        except Exception as e:
            print('Exception:',e)

In [3]:
import numpy as np

data=np.array(data)/255.0
data=np.reshape(data,(data.shape[0],img_size,img_size,1))
target=np.array(target)

from keras.utils import np_utils

new_target=np_utils.to_categorical(target)

Using TensorFlow backend.


In [4]:
from keras.models import Sequential
from keras.layers import Dense,Activation,Flatten,Dropout
from keras.layers import Conv2D,MaxPooling2D
from keras.callbacks import ModelCheckpoint

In [5]:
model = Sequential()

In [6]:
model.add(Conv2D(200,(3,3),input_shape=data.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

In [7]:
model.add(Conv2D(100,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

In [8]:
model.add(Flatten())
model.add(Dropout(0.5))

In [9]:
model.add(Dense(50,activation='relu'))

In [10]:
model.add(Dense(2,activation='softmax'))

In [11]:
model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [12]:
from sklearn.model_selection import train_test_split
train_data,test_data,train_target,test_target=train_test_split(data,target,test_size=0.1)

In [13]:
checkpoint = ModelCheckpoint('model-{epoch:03d}.model',monitor='val_loss',verbose=0,save_best_only=True,mode='auto')
history=model.fit(train_data,train_target,epochs=20,callbacks=[checkpoint],validation_split=0.2)


Train on 990 samples, validate on 248 samples
Epoch 1/20
990/990 [==============================] - 282s 284ms/step - loss: 0.7267 - accuracy: 0.5667 - val_loss: 0.6091 - val_accuracy: 0.6895
Epoch 2/20
990/990 [==============================] - 180s 182ms/step - loss: 0.4726 - accuracy: 0.7727 - val_loss: 0.4623 - val_accuracy: 0.7823
Epoch 3/20
990/990 [==============================] - 178s 180ms/step - loss: 0.3048 - accuracy: 0.8727 - val_loss: 0.3014 - val_accuracy: 0.9073
Epoch 4/20
990/990 [==============================] - 179s 181ms/step - loss: 0.2147 - accuracy: 0.9202 - val_loss: 0.2397 - val_accuracy: 0.9274
Epoch 5/20
990/990 [==============================] - 179s 181ms/step - loss: 0.1472 - accuracy: 0.9515 - val_loss: 0.2143 - val_accuracy: 0.9274
Epoch 6/20
990/990 [==============================] - 179s 181ms/step - loss: 0.1309 - accuracy: 0.9576 - val_loss: 0.1904 - val_accuracy: 0.9435
Epoch 7/20
990/990 [==============================] - 177s 179ms/step - loss: 

In [14]:
model.evaluate(test_data,test_target)

138/138 [==============================] - 8s 57ms/step


[0.07807066016813632, 0.9637681245803833]

In [17]:
import cv2
face_clsfr=cv2.CascadeClassifier('Videos/facemask/face-mask-detection-keras-master/haarcascade_frontalface_default.xml')

source=cv2.VideoCapture(0)

labels_dict={0:'MASK',1:'NO MASK'}
color_dict={0:(0,255,0),1:(0,0,255)}

In [18]:
while(True):

    ret,img=source.read()
    gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces=face_clsfr.detectMultiScale(gray,1.3,5)  

    for (x,y,w,h) in faces:
    
        face_img=gray[y:y+w,x:x+w]
        resized=cv2.resize(face_img,(100,100))
        normalized=resized/255.0
        reshaped=np.reshape(normalized,(1,100,100,1))
        result=model.predict(reshaped)
        print(result)
        label=np.argmax(result,axis=1)[0]
        print(label)
      
        cv2.rectangle(img,(x,y),(x+w,y+h),color_dict[label],0)
        cv2.rectangle(img,(x,y-40),(x+w,y),color_dict[label],-1)
        cv2.putText(img, labels_dict[label], (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),0)
        
        
    cv2.imshow('LIVE',img)
    key=cv2.waitKey(1)
    
    if(key==27):
        break
        
cv2.destroyAllWindows()
source.release()

[[0.03851342 0.9614866 ]]
1
[[6.3782977e-04 9.9936217e-01]]
1
[[6.0160703e-04 9.9939835e-01]]
1
[[0.00206476 0.9979353 ]]
1
[[2.6434771e-04 9.9973565e-01]]
1
[[3.692989e-04 9.996307e-01]]
1
[[2.6920435e-04 9.9973077e-01]]
1
[[1.8340471e-04 9.9981660e-01]]
1
[[4.093434e-04 9.995907e-01]]
1
[[5.208560e-04 9.994791e-01]]
1
[[4.9978861e-04 9.9950016e-01]]
1
[[2.3518494e-04 9.9976486e-01]]
1
[[2.3997886e-04 9.9975997e-01]]
1
[[2.1772037e-04 9.9978226e-01]]
1
[[2.1657543e-04 9.9978346e-01]]
1
[[2.4479046e-04 9.9975520e-01]]
1
[[2.764234e-04 9.997236e-01]]
1
[[2.592336e-04 9.997408e-01]]
1
[[2.8243900e-04 9.9971753e-01]]
1
[[2.4425238e-04 9.9975580e-01]]
1
[[2.5411934e-04 9.9974591e-01]]
1
[[2.7673878e-04 9.9972326e-01]]
1
[[3.2401545e-04 9.9967599e-01]]
1
[[4.0754105e-04 9.9959248e-01]]
1
[[2.5867246e-04 9.9974138e-01]]
1
[[2.6679586e-04 9.9973315e-01]]
1
[[2.2329537e-04 9.9977666e-01]]
1
[[2.8991827e-04 9.9971002e-01]]
1
[[2.6803449e-04 9.9973196e-01]]
1
[[3.0068168e-04 9.9969935e-01]]
1
[[